## Prepare data reader for a given chain and date

In [1]:
from op_analytics.coreutils.duckdb_inmem import init_client
from op_analytics.coreutils.partitioned.reader import DataReader
from op_analytics.coreutils.partitioned.location import DataLocation
from op_analytics.datapipeline.etl.intermediate.construct import construct_data_readers

from op_analytics.datapipeline.models.compute.udfs import create_duckdb_macros

model_name = "refined_traces_models"

# Define the input data range.
read_batches: list[DataReader] = construct_data_readers(
    chains=["op"],
    models=[model_name],
    range_spec="@20241118:+1",
    read_from=DataLocation.GCS
)


# Select input for one date and build the intermediate model inputs.
batch = read_batches[0]


duckdb_client = init_client()
create_duckdb_macros(duckdb_client)


2024-12-14 22:09:13 [debug    ] connecting to OPLABS Clickhouse client... filename=client.py lineno=25 process=46910
2024-12-14 22:09:13 [info     ] loaded vault from .env file    filename=vault.py lineno=32 process=46910
2024-12-14 22:09:13 [debug    ] loaded vault: 17 items         filename=vault.py lineno=76 process=46910
2024-12-14 22:09:14 [debug    ] initialized OPLABS Clickhouse client. filename=client.py lineno=37 process=46910
2024-12-14 22:09:14 [info     ] prepared 1 input batches.      filename=bydate.py lineno=96 process=46910


## Run the model

This automatically registers the model outputs as duckdb tables.

In [2]:
from op_analytics.datapipeline.models.compute.testutils import execute_model_in_memory

execute_model_in_memory(
    duckdb_client=duckdb_client,
    model=model_name,
    data_reader=batch,
    limit_input_parquet_files=1
)

# The duckdb database will have the following:
#   - input tables
#   - views used by the model
#   - model outputs
# 
# You can use duckdb to inspect any of the above results.
duckdb_client.sql("SHOW TABLES")

2024-12-14 22:09:14 [info     ] Executing model function...    filename=testutils.py lineno=220 process=46910
2024-12-14 22:09:14 [info     ] duckdb dataset='ingestion/transactions_v1' using 1/22 parquet paths, first path is gs://oplabs-tools-data-sink/ingestion/transactions_v1/chain=op/dt=2024-11-18/000128144000.parquet filename=reader.py lineno=68 process=46910
2024-12-14 22:09:15 [info     ] registered view: 'ingestion_transactions_v1' using 1 parquet paths filename=client.py lineno=53 process=46910
2024-12-14 22:09:15 [info     ] duckdb dataset='ingestion/blocks_v1' using 1/22 parquet paths, first path is gs://oplabs-tools-data-sink/ingestion/blocks_v1/chain=op/dt=2024-11-18/000128144000.parquet filename=reader.py lineno=68 process=46910
2024-12-14 22:09:16 [info     ] registered view: 'ingestion_blocks_v1' using 1 parquet paths filename=client.py lineno=53 process=46910
2024-12-14 22:09:16 [info     ] duckdb dataset='ingestion/traces_v1' using 1/22 parquet paths, first path is gs:

┌───────────────────────────────────────────────┐
│                     name                      │
│                    varchar                    │
├───────────────────────────────────────────────┤
│ daily_trace_calls_agg_to_v1                   │
│ daily_trace_calls_agg_tr_to                   │
│ daily_trace_calls_agg_tr_to_tx_to             │
│ daily_trace_calls_agg_tr_to_tx_to_v1          │
│ ingestion_blocks_v1                           │
│ ingestion_traces_v1                           │
│ ingestion_transactions_v1                     │
│ refined_trace_calls                           │
│ refined_trace_calls_agg_to_hash_v1            │
│ refined_trace_calls_agg_tr_from_tr_to_hash    │
│ refined_trace_calls_agg_tr_from_tr_to_hash_v1 │
│ refined_trace_calls_agg_tr_to_hash            │
│ refined_trace_calls_v1                        │
│ refined_transactions_fees                     │
├───────────────────────────────────────────────┤
│                    14 rows                    │


## Verify model results - Traces

In [3]:
duckdb_client.sql("SELECT * FROM daily_trace_calls_agg_to_v1 ORDER BY count_transactions_called DESC LIMIT 10")

┌────────────┬─────────┬─────────┬──────────┬────────────────────────────────────────────┬───────────────────────────┬──────────────────────────────────────┬────────────────────────────┬────────────────────────────────┬──────────────────────────────┬────────────────────────────┬───────────────────────────┬──────────────────────────────────────┬──────────────────────────┬─────────────────────────────────────┬───────────────────────┬─────────────────────────────┬───────────────────────┬─────────────────────────────┬───────────────────────────┬──────────────────────────────────────┬───────────────────────┬──────────────────────────────────┬───────────────────────────────┬──────────────────────────────────────────┬─────────────────────────────────┬──────────────────────────────────────────┬──────────────────────────────────────────────┬──────────────────────────────────────┬───────────────────────────────────────────┬─────────────────────────────────┬────────────────────────────────┬──────

### Test Transaction Metrics

In [4]:
duckdb_client.sql("""
                  SELECT trace_to_address,
                    count_transactions_called_with_internal_type_call, count_transactions_called_with_internal_type_call_or_delegate,
                    count_transactions_called
                  FROM daily_trace_calls_agg_to_v1
                  ORDER BY count_transactions_called_with_internal_type_call DESC
                  LIMIT 10
                  """)

┌────────────────────────────────────────────┬───────────────────────────────────────────────────┬───────────────────────────────────────────────────────────────┬───────────────────────────┐
│              trace_to_address              │ count_transactions_called_with_internal_type_call │ count_transactions_called_with_internal_type_call_or_delegate │ count_transactions_called │
│                  varchar                   │                       int64                       │                             int64                             │           int64           │
├────────────────────────────────────────────┼───────────────────────────────────────────────────┼───────────────────────────────────────────────────────────────┼───────────────────────────┤
│ 0x4200000000000000000000000000000000000006 │                                              8191 │                                                          8264 │                      8320 │
│ 0xded3b9a8dbedc2f9cb725b55d0e686a81e6d06dc 

### Test Gas Used Metrics

In [5]:
duckdb_client.sql("""
                  SELECT
                    trace_to_address,
                  
                    sum_trace_gas_used_minus_subtraces_tx_success_called_with_internal_type_call,
                    sum_tx_l2_gas_used_amortized_by_call_tx_success_called_with_internal_type_call,
                  
                    sum_tx_l2_fee_native_minus_subtraces_tx_success_called_with_internal_type_call
                    sum_tx_l2_fee_native_amortized_by_call_tx_success_called_with_internal_type_call,
                    sum_tx_fee_native_amortized_by_call_tx_success_called_with_internal_type_call,
                  
                    count_transactions_called_with_internal_type_call,
                    count_transactions_called_with_internal_type_call_or_delegate,
                    count_transactions_called
                  
                  FROM daily_trace_calls_agg_to_v1
                  ORDER BY sum_trace_gas_used_minus_subtraces_tx_success_called_with_internal_type_call DESC
                  LIMIT 10
                  """)

┌────────────────────────────────────────────┬──────────────────────────────────────────────────────────────────────────────┬────────────────────────────────────────────────────────────────────────────────┬──────────────────────────────────────────────────────────────────────────────────┬───────────────────────────────────────────────────────────────────────────────┬───────────────────────────────────────────────────┬───────────────────────────────────────────────────────────────┬───────────────────────────┐
│              trace_to_address              │ sum_trace_gas_used_minus_subtraces_tx_success_called_with_internal_type_call │ sum_tx_l2_gas_used_amortized_by_call_tx_success_called_with_internal_type_call │ sum_tx_l2_fee_native_amortized_by_call_tx_success_called_with_internal_type_call │ sum_tx_fee_native_amortized_by_call_tx_success_called_with_internal_type_call │ count_transactions_called_with_internal_type_call │ count_transactions_called_with_internal_type_call_or_delegate │ 

In [6]:
duckdb_client.sql("""
    SELECT * FROM refined_trace_calls_v1
                  where transaction_hash = '0xc620133c2339f36d8bfae889ea29e9986a70182f7bbe3380d0622f3801619eda'
                  AND block_number = 128145924
                  ORDER BY trace_address ASC
                  LIMIT 10
                  """)

┌────────────┬─────────┬──────────┬─────────┬──────────────┬─────────────────┬────────────────────────────────────────────────────────────────────┬───────────────────┬────────────────────────────────────────────┬────────────────────────────────────────────┬─────────────────┬────────────────┬───────────────┬──────────────┬────────────────┬────────────────────────┬───────────────────┬────────────────────────────────────────────┬────────────────────────────────────────────┬───────────────────────┬───────────────────────┬───────────────────────┬───────────────────────────┬───────────────────────┬───────────────────────────────┬──────────────────────┬───────────────────────────┬───────────────────────────────┬───────────────────────────────────┬───────────────────────────┬────────────────────────────────┬──────────────────────┬─────────────────────┬────────────────────────┬────────────────────────┬────────────────────────┬──────────────┬─────────────────────┬────────────┬─────────┬─────────

### Check the data output size

In [7]:
duckdb_client.sql("SELECT COUNT(*) AS interm_num_calls, COUNT(DISTINCT transaction_hash) AS num_txs, COUNT(DISTINCT trace_to_address) AS num_trace_tos FROM refined_trace_calls_v1")

┌──────────────────┬─────────┬───────────────┐
│ interm_num_calls │ num_txs │ num_trace_tos │
│      int64       │  int64  │     int64     │
├──────────────────┼─────────┼───────────────┤
│          1274668 │   33166 │         43972 │
└──────────────────┴─────────┴───────────────┘

In [8]:
duckdb_client.sql("SELECT COUNT(*) AS num_rows, COUNT(DISTINCT transaction_hash) AS num_txs, COUNT(DISTINCT trace_to_address) AS num_trace_tos FROM refined_trace_calls_agg_to_hash_v1")

┌──────────┬─────────┬───────────────┐
│ num_rows │ num_txs │ num_trace_tos │
│  int64   │  int64  │     int64     │
├──────────┼─────────┼───────────────┤
│   528958 │   33166 │         43972 │
└──────────┴─────────┴───────────────┘

In [9]:
duckdb_client.sql("SELECT COUNT(*) AS num_rows, COUNT(DISTINCT trace_to_address) AS num_trace_tos FROM daily_trace_calls_agg_to_v1")

┌──────────┬───────────────┐
│ num_rows │ num_trace_tos │
│  int64   │     int64     │
├──────────┼───────────────┤
│    43966 │         43966 │
└──────────┴───────────────┘

### Get table schema

In [10]:
duckdb_client.sql("DESCRIBE daily_trace_calls_agg_to_v1")


┌──────────────────────────────────────────────────────────────────────────────────┬────────────────┬─────────┬─────────┬─────────┬─────────┐
│                                   column_name                                    │  column_type   │  null   │   key   │ default │  extra  │
│                                     varchar                                      │    varchar     │ varchar │ varchar │ varchar │ varchar │
├──────────────────────────────────────────────────────────────────────────────────┼────────────────┼─────────┼─────────┼─────────┼─────────┤
│ dt                                                                               │ DATE           │ YES     │ NULL    │ NULL    │ NULL    │
│ chain                                                                            │ VARCHAR        │ YES     │ NULL    │ NULL    │ NULL    │
│ network                                                                          │ VARCHAR        │ YES     │ NULL    │ NULL    │ NULL    │
│ chai